In [ ]:
from data import *
from models import *
from utils import *

%load_ext autoreload
%autoreload 2 # %autoreload 0. disables autoloading. 2 enables autoreload ALL

import torch
import torch.optim as torch_optimizer # by naming it optimizer, it will conflict with the optimizer instantiated class
from torchinfo import summary

In [ ]:
# parameters
RANDOM_SEED = 42
LEARNING_RATE = 0.01
BATCH_SIZE = 64
N_EPOCHS = 15

IMG_SIZE = 32
N_CLASSES = 10

## 1. Data & DataLoader

In [ ]:
"Data Loaders"
kwargs = {'batch_size': BATCH_SIZE, 'shuffle': True}

train_dataloader = torch.utils.data.DataLoader(train_dataset, **kwargs)
test_dataloader = torch.utils.data.DataLoader(test_dataset, **kwargs)


import matplotlib.pyplot as plt
batch_data, batch_label = next(iter(train_dataloader)) 

fig = plt.figure()
for i in range(12):
  plt.subplot(3,4,i+1)
  plt.tight_layout()
  plt.imshow(batch_data[i].squeeze(0), cmap='gray')
  plt.title(batch_label[i].item())
  plt.xticks([])
  plt.yticks([])

## 2. Model & Model Parameters

In [ ]:
model = FirstDNN()

!pip install torchinfo
from torchinfo import summary
summary(model,input_size=(1,28,28),verbose=2);

## 3. Model Training

In [ ]:

model = FirstDNN()
errorFun = torch.nn.functional.nll_loss
optimizer = torch_optimizer.SGD(model.parameters(), lr=0.01, momentum=0.9)
device = torch.device("mps")

In [ ]:
class reshape(nn.Module):
    def __init__(self,pixels):
        super().__init__()
        self.pixels = pixels
    def forward(self,x):
        x = x.view(-1,self.pixels)
        return x

container = {                          				
            "conv1": nn.Conv2d(1,32,(3,3),1,1),     
            "relu": nn.ReLU(), 
            "conv2": nn.Conv2d(32,32,(3,3),1,1),    
            "relu": nn.ReLU(),
                                                    
            "pool1": nn.MaxPool2d((2,2),2,0),       

            "conv3": nn.Conv2d(32,32,(3,3),1,1),   
            "relu": nn.ReLU(),
            "conv4": nn.Conv2d(32,32,(3,3),1,1),  
            "relu": nn.ReLU(),
                                                    
            "pool2": nn.MaxPool2d((2,2),2,0),       
            
            "conv5": nn.Conv2d(32,32,(3,3),1,0),  
            "relu": nn.ReLU(),
            "conv6": nn.Conv2d(32,32,(3,3),1,0),  
            "relu": nn.ReLU(),

			"conv7" : nn.Conv2d(32, 10, 3),
			"reshaper": reshape(10),
			"log_softmax": nn.LogSoftmax(dim=1),  
            

            # "reshaper": reshape(1024*9),
            # "fc1": nn.Linear(1024*3*3,50),               
            # "relu": nn.ReLU(),
            # "fc2": nn.Linear(50,10),                
            # "softmax": nn.LogSoftmax(dim=1),           
            # Using softmax to check if probability sum == 1

        }
new_model = nn.Sequential(*container.values())
summary(new_model,input_size=(1,28,28),verbose=2);

In [ ]:
for epoch in range(1, 5):
	train(train_dataloader, new_model, errorFun, optimizer,epoch)
	test(test_dataloader, new_model, errorFun)

## 4. Learning Curves

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(15,10))
axs[0, 0].plot(training_losses_epochwise)
axs[0, 0].set_title("Training Loss")
axs[1, 0].plot(training_accuracy_epochwise)
axs[1, 0].set_title("Training Accuracy")
axs[0, 1].plot(test_losses_epochwise)
axs[0, 1].set_title("Test Loss")
axs[1, 1].plot(test_accuracy_epochwise)
axs[1, 1].set_title("Test Accuracy")